In [1]:
import pandas as pd

df = pd.read_csv("portfolio_articles_2.csv", sep="|", dtype='str')

In [14]:
df

,url,title,content,date,full_text
0,https://www.portfolio.hu/gazdasag/20220729/bom...,Bombameglepetés: bivalyerős a spanyol gazdaság...,Közzétették a második negyedéves spanyol GDP-a...,2022/07/29,Bombameglepetés: bivalyerős a spanyol gazdaság...
1,https://www.portfolio.hu/gazdasag/20220729/jol...,Jól teljesített a francia gazdaság a második n...,A várt felett nőtt a francia gazdaság a másodi...,2022/07/29,Jól teljesített a francia gazdaság a második n...
2,https://www.portfolio.hu/gazdasag/20220729/orb...,"Orbán Viktor: Ukrajnát nem lehet megvédeni, há...",Megbukott a nyugati világ háborús stratégiája ...,2022/07/29,"Orbán Viktor: Ukrajnát nem lehet megvédeni, há..."
3,https://www.portfolio.hu/gazdasag/20220729/fat...,Fatüzelésűre állítaná az iskolákat a magyar állam,"""Haladéktalanul fel kell mérni az intézményekb...",2022/07/29,Fatüzelésűre állítaná az iskolákat a magyar ál...
4,https://www.portfolio.hu/gazdasag/20220728/mar...,Már Líbiában lehet a lopott ukrán árpa és búza,Oroszország által kifosztott ukrán készletekbő...,2022/07/28,Már Líbiában lehet a lopott ukrán árpa és búza...
...,...,...,...,...,...
20602,https://www.portfolio.hu/gazdasag/20200616/uja...,Újabb német autóipari multi halasztja el magya...,Az autóipari kilátások bizonytalansága miatt 1...,2020/06/16,Újabb német autóipari multi halasztja el magya...
20603,https://www.portfolio.hu/gazdasag/20200616/ami...,"Amilyen gyors volt a beszakadás, olyan hirtele...","Az áprilisi 14,7 százalékos zuhanást követően ...",2020/06/16,"Amilyen gyors volt a beszakadás, olyan hirtele..."
20604,https://www.portfolio.hu/gazdasag/20200616/kor...,Koronavírus: felfüggesztették Norvégiában a ko...,Adatvédelmi problémák miatt felfüggesztik a ko...,2020/06/16,Koronavírus: felfüggesztették Norvégiában a ko...
20605,https://www.portfolio.hu/gazdasag/20200616/kes...,Készültségben a dél-koreai hadsereg az észak-k...,Fokozott készültségbe helyezték a dél-koreai h...,2020/06/16,Készültségben a dél-koreai hadsereg az észak-k...


In [7]:
df = df[
    (df["title"] != "Title not found") & 
    ~df["content"].fillna("").str.contains("Content not found") & 
    ~df["content"].fillna("").str.strip().eq("") & 
    ~df["content"].fillna("").str.contains("Error fetching content") & 
    df["error"].isnull()
]

In [9]:
df = df.drop("error", axis=1)
df['full_text'] = df['title'] + ". " + df['content']
df = df.reset_index(drop=True)

In [4]:
df[~df["error"].isnull()]

,url,title,content,date,error
2646,https://www.portfolio.hu/gazdasag/20220409/sp-...,NaN,NaN,NaN,"HTTPSConnectionPool(host='www.portfolio.hu', p..."
7962,https://www.portfolio.hu/gazdasag/20211003/veg...,NaN,NaN,NaN,"HTTPSConnectionPool(host='www.portfolio.hu', p..."
8589,https://www.portfolio.hu/gazdasag/20210909/uj-...,NaN,NaN,NaN,"HTTPSConnectionPool(host='www.portfolio.hu', p..."
8747,https://www.portfolio.hu/gazdasag/20210814/oly...,NaN,NaN,NaN,404 Client Error: Not Found for url: https://w...
9885,https://www.portfolio.hu/cimke/negyedik%20hull...,NaN,NaN,NaN,404 Client Error: Not Found for url: https://w...
10784,../../../gazdasag/20210614/pleschinger-gyula-a...,NaN,NaN,NaN,Invalid URL '../../../gazdasag/20210614/plesch...
18265,https://www.portfolio.hu/gazdasag/20200921/az-...,NaN,NaN,NaN,"HTTPSConnectionPool(host='www.portfolio.hu', p..."


In [11]:
df.to_csv("portfolio_articles_2.csv", sep="|", encoding='utf-8', index=False)

In [16]:
import pandas as pd
import asyncio
from concurrent.futures import ThreadPoolExecutor
from google import genai
from google.genai import types

client = genai.Client(api_key="")

# Function to translate each text
def translate_text(text):
    try:
        response = client.models.generate_content(
            model="gemini-2.0-flash",
            config=types.GenerateContentConfig(
                system_instruction="You are a translator that translates Hungarian text to English."
            ),
            contents=f"Please translate the following article to English in simple text format: {text}"
        )
        return response.text
    except Exception as e:
        print(f"Error: {e}")
        return None

# Asynchronous wrapper for the translate function
async def translate_batch_async(batch):
    loop = asyncio.get_event_loop()
    with ThreadPoolExecutor() as executor:
        tasks = [loop.run_in_executor(executor, translate_text, text) for text in batch["full_text"]]
        results = await asyncio.gather(*tasks)
    batch["translated_content"] = results
    return batch

# Main processing loop
async def main(df):
    try:
        df_translated = pd.read_csv("portfolio_articles_2_translated.csv", sep="|", encoding='utf-8')
        processed_ids = set(df_translated.index)
    except FileNotFoundError:
        df_translated = pd.DataFrame(columns=["full_text", "translated_content"])
        processed_ids = set()

    batch_size = 75  # Number of rows to process per batch
    start_index = max(processed_ids) + 1 if processed_ids else 0

    for i in range(start_index, len(df), batch_size):
        batch = df.iloc[i:i + batch_size]
        batch = await translate_batch_async(batch)
        
        # Append to the result DataFrame
        df_translated = pd.concat([df_translated, batch], ignore_index=True)
        
        # Save to CSV after each batch
        df_translated.to_csv("portfolio_articles_2_translated.csv", index=False, sep="|", encoding='utf-8')
        
        print(f"Batch {i // batch_size + 1} processed and saved. Total articles translated: {len(df_translated)}")


# Run the asynchronous main function
await main(df)

C:\Users\tothk\AppData\Local\Temp\ipykernel_14048\3663119335.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batch["translated_content"] = results


Batch 266 processed and saved. Total articles translated: 19950


C:\Users\tothk\AppData\Local\Temp\ipykernel_14048\3663119335.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batch["translated_content"] = results


Batch 267 processed and saved. Total articles translated: 20025


C:\Users\tothk\AppData\Local\Temp\ipykernel_14048\3663119335.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batch["translated_content"] = results


...

Batch 273 processed and saved. Total articles translated: 20475


C:\Users\tothk\AppData\Local\Temp\ipykernel_14048\3663119335.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batch["translated_content"] = results


Batch 274 processed and saved. Total articles translated: 20550


C:\Users\tothk\AppData\Local\Temp\ipykernel_14048\3663119335.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batch["translated_content"] = results


Batch 275 processed and saved. Total articles translated: 20607


In [17]:
df_proba = pd.read_csv("portfolio_articles_2_translated.csv", sep="|", dtype='str')

In [18]:
df_proba[df_proba["translated_content"].isnull()].count()

full_text             273
translated_content      0
url                   273
title                 273
content               273
date                  273
dtype: int64

In [21]:
# Function to translate each text 
def translate_text(text):
    try:
        response = client.models.generate_content(
            model="gemini-2.0-flash",
            config=types.GenerateContentConfig(
                system_instruction="You are a translator that translates Hungarian text to English."
            ),
            contents=f"Please translate the following article to English in simple text format: {text}"
        )
        return response.text
    except Exception as e:
        print(f"Error: {e}")
        return None

# Asynchronous wrapper for the translate function
async def translate_batch_async(batch):
    loop = asyncio.get_event_loop()
    with ThreadPoolExecutor() as executor:
        tasks = [loop.run_in_executor(executor, translate_text, text) for text in batch["full_text"]]
        results = await asyncio.gather(*tasks)
    batch["translated_content"] = results
    return batch

# Main function to process and retry NaN translations
async def main():
    try:
        df_translated = pd.read_csv("portfolio_articles_2_translated.csv", sep="|", encoding='utf-8')
    except FileNotFoundError:
        df_translated = pd.DataFrame(columns=["full_text", "translated_content"])
    
    # First pass: Process untranslated rows
    untranslated_rows = df_translated["translated_content"].isna().sum()
    print(f"Untranslated rows found: {untranslated_rows}")

    if untranslated_rows > 0:
        to_retry = df_translated[df_translated["translated_content"].isna()]
        df_translated.loc[to_retry.index, "translated_content"] = (await translate_batch_async(to_retry))["translated_content"].values

        # Save the corrected translations
        df_translated.to_csv("portfolio_articles_2_translated.csv", index=False, sep="|", encoding='utf-8')
        print(f"Retry complete! Remaining untranslated rows: {df_translated['translated_content'].isna().sum()}")
    
    print("Translation process complete!")

# Run the asynchronous main function
await main()


Untranslated rows found: 133


C:\Users\tothk\AppData\Local\Temp\ipykernel_14048\1523488885.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batch["translated_content"] = results


Retry complete! Remaining untranslated rows: 111
Translation process complete!


In [22]:
df_proba = pd.read_csv("portfolio_articles_2_translated.csv", sep="|", dtype='str')

df_proba.dropna(subset=['translated_content'], inplace=True)
df_proba.reset_index(drop=True, inplace=True)

df_proba.to_csv("portfolio_articles_2_translated.csv", index=False, sep="|", encoding='utf-8')